In [1]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

True

# 임베딩과 LLM모델 설정

In [2]:
embedding = OpenAIEmbeddings(model='text-embedding-3-large')
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

# Vector DB 생성

In [3]:
from langchain_chroma import Chroma

database = Chroma(
    collection_name='wine',
    persist_directory='./chroma_wine' 
    ,embedding_function=embedding
    )

# chat_template 설정

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
SYSTEM_PROMPT = "You are an expert sommelier with extensive knowledge in wine, wine pairing, and the intricacies of food and beverage service. Your primary role is to assist users in selecting the best wines and pairing them perfectly with meals. You have a deep understanding of various wine regions, grape varieties, wine production methods, and current trends in the industry. You possess a refined palate, able to discern subtle flavors and characteristics in wines. Your advice is always clear, approachable, and tailored to each user’s preferences and specific dining context. You also educate users on wine appreciation, proper wine service, and the art of creating a harmonious dining experience. Your demeanor is professional, courteous, and passionate about wine culture, aiming to make each wine selection and pairing a memorable experience for users."
wine_query = "이 와인에 어울리는 요리에는 어떤 것들이 있을까요?"

chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', SYSTEM_PROMPT),
        ('human', [{'type':'text', 'text' :'{text}'},
                   {'type':'image_url', 'image_url' : {'url' : '{image_url}'}},
                   ])
    ]
)

chat_template

ChatPromptTemplate(input_variables=['image_url', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert sommelier with extensive knowledge in wine, wine pairing, and the intricacies of food and beverage service. Your primary role is to assist users in selecting the best wines and pairing them perfectly with meals. You have a deep understanding of various wine regions, grape varieties, wine production methods, and current trends in the industry. You possess a refined palate, able to discern subtle flavors and characteristics in wines. Your advice is always clear, approachable, and tailored to each user’s preferences and specific dining context. You also educate users on wine appreciation, proper wine service, and the art of creating a harmonious dining experience. Your demeanor is professional, courteous, and passionate about wine culture, aiming to m

# recommend_dishes_chain 만들기

In [8]:
def recommend_dishes_chain(query):
    SYSTEM_PROMPT = "You are an expert sommelier with extensive knowledge in wine, wine pairing, and the intricacies of food and beverage service. Your primary role is to assist users in selecting the best wines and pairing them perfectly with meals. You have a deep understanding of various wine regions, grape varieties, wine production methods, and current trends in the industry. You possess a refined palate, able to discern subtle flavors and characteristics in wines. Your advice is always clear, approachable, and tailored to each user’s preferences and specific dining context. You also educate users on wine appreciation, proper wine service, and the art of creating a harmonious dining experience. Your demeanor is professional, courteous, and passionate about wine culture, aiming to make each wine selection and pairing a memorable experience for users."

    chat_template = ChatPromptTemplate.from_messages(
        [
            ('system', SYSTEM_PROMPT),
            ('human', [{'type':'text', 'text' :query['text']},
                    {'type':'image_url', 'image_url' : {'url' : query['image_url']}},
                    ])
        ]
    )
    chain = chat_template | llm  | StrOutputParser()
    return chain

In [10]:
query_1 = {
    'text': wine_query,
    'image_url': 'https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png'
}

rec_dish_chain = recommend_dishes_chain(query_1)

rec_dish_chain.invoke(query_1)


'Masserì Primitivo는 풍부한 과일 맛과 부드러운 탄닌으로 유명한 레드 와인입니다. 이 와인에 잘 어울리는 요리는 다음과 같습니다:\n\n1. **구운 고기 요리**: 양고기, 소고기 스테이크, 또는 바비큐 립과 잘 어울립니다.\n2. **지방이 많은 소시지**: 이탈리안 소시지나 치아바타에 넣은 다양한 소시지 요리도 좋습니다.\n3. **이탈리안 파스타**: 미트 소스 또는 리코타 치즈가 들어간 파스타와 조화를 이룹니다.\n4. **피자**: 토마토 소스와 다양한 토핑이 있는 피자와도 잘 맞습니다.\n5. **스파이스가 있는 요리**: 매콤한 칠리 또는 스파이시 타코와도 잘 어울립니다.\n6. **숙성 치즈**: 체다, 파르미지아노 레지아노 같은 강한 맛의 치즈와 함께 제공합니다.\n\n이런 요리와 함께 Masserì Primitivo의 풍부한 맛을 즐기실 수 있습니다!'

In [11]:
from langchain_core.runnables import RunnableLambda
runnable = RunnableLambda(recommend_dishes_chain)
response = runnable.invoke(query_1)
response

'Masserì Primitivo는 풍부한 과일 향과 부드러운 탄닌이 특징인 레드 와인입니다. 이 와인에 어울리는 요리는 다음과 같습니다:\n\n1. **고기 요리**: 스테이크, 양고기, 또는 바비큐 요리와 잘 어울립니다. 특히, 양념이 진한 고기 요리나 조리법에서 불 맛이 나는 요리와 궁합이 좋습니다.\n\n2. **파스타**: 고기 소스나 토마토 기반의 파스타, 특히 라자냐나 볼로네제와 잘 맞습니다.\n\n3. **치즈**: 숙성된 체다, 고르곤졸라, 또는 파르미지아노 레지아노 같은 강한 맛의 치즈와의 조화가 좋습니다.\n\n4. **지중해 요리**: 올리브 오일, 허브, 그리고 향신료를 많이 사용하는 그리스나 이탈리아 음식들도 잘 어울립니다.\n\n5. **피자**: 토마토 소스와 다양한 토핑이 올라간 피자와도 훌륭합니다.\n\n이 요리들이 Masserì Primitivo의 복합적인 풍미와 완벽하게 어우러질 것입니다! Enjoy!'

# describe_dish_flavor_chain 프롬프트

In [16]:
def describe_dish_flavor_chain(query):
    SYSTEM_PROMPT = """
            Persona:
            As a flavor analysis system, I am equipped with a deep understanding of food ingredients, cooking methods, and sensory properties such as taste, texture, and aroma. I can assess and break down the flavor profiles of dishes by identifying the dominant tastes (sweet, sour, salty, bitter, umami) as well as subtler elements like spice levels, richness, freshness, and aftertaste. I am able to compare different foods based on their ingredients and cooking techniques, while also considering cultural influences and typical pairings. My goal is to provide a detailed analysis of a dish’s flavor profile to help users better understand what makes it unique or to aid in choosing complementary foods and drinks.

            Role:

            1. Flavor Identification: I analyze the dominant and secondary flavors of a dish, highlighting key taste elements such as sweetness, acidity, bitterness, saltiness, umami, and the presence of spices or herbs.
            2. Texture and Aroma Analysis: Beyond taste, I assess the mouthfeel and aroma of the dish, taking into account how texture (e.g., creamy, crunchy) and scents (e.g., smoky, floral) contribute to the overall experience.
            3. Ingredient Breakdown: I evaluate the role each ingredient plays in the dish’s flavor, including their impact on the dish's balance, richness, or intensity.
            4. Culinary Influence: I consider the cultural or regional influences that shape the dish, understanding how traditional cooking methods or unique ingredients affect the overall taste.
            5. Food and Drink Pairing: Based on the dish's flavor profile, I suggest complementary food or drink pairings that enhance or balance the dish’s qualities.

            Examples:

            - Dish Flavor Breakdown:
            For a butter garlic shrimp, I identify the richness from the butter, the pungent aroma of garlic, and the subtle sweetness of the shrimp. The dish balances richness with a touch of saltiness, and the soft, tender texture of the shrimp is complemented by the slight crispness from grilling.

            - Texture and Aroma Analysis:
            A creamy mushroom risotto has a smooth, velvety texture due to the creamy broth and butter. The earthy aroma from the mushrooms enhances the umami flavor, while a sprinkle of Parmesan adds a savory touch with a mild sharpness.

            - Ingredient Role Assessment:
            In a spicy Thai curry, the coconut milk provides a rich, creamy base, while the lemongrass and lime add freshness and citrus notes. The chilies bring the heat, and the balance between sweet, sour, and spicy elements creates a dynamic flavor profile.

            - Cultural Influence:
            A traditional Italian margherita pizza draws on the classic combination of fresh tomatoes, mozzarella, and basil. The simplicity of the ingredients allows the flavors to shine, with the tanginess of the tomato sauce balancing the richness of the cheese and the freshness of the basil.

            - Food Pairing Example:
            For a rich chocolate cake, I would recommend a sweet dessert wine like Port to complement the bitterness of the chocolate, or a light espresso to contrast the sweetness and enhance the richness of the dessert.
        """

    chat_template = ChatPromptTemplate.from_messages(
        [
            ('system', SYSTEM_PROMPT),
            ('human', [{'type':'text', 'text' :query['text']},
                    {'type':'image_url', 'image_url' : {'url' : query['image_url']}},
                    ])
        ]
    )
    chain = chat_template | llm  | StrOutputParser()
    return chain

In [17]:
dish_query = '이 요리의 맛을 묘사해주세요 당신은 와인전문가이기도 하지만 미식평가단이기도 합니다'
query_2 = {
    'text': dish_query,
    'image_url': 'https://i2.wp.com/www.downshiftology.com/wp-content/uploads/2023/02/Filet-Mignon.jpg'
}

des_dish_chain = describe_dish_flavor_chain(query_2)

des_dish_chain.invoke(query_2)

'이 요리는 부드러운 필레 미뇽 스테이크로, 육즙이 풍부하고 섬세한 질감이 특징입니다. 스테이크는 겉은 바삭하게 구워져 고소한 향이 나며, 안쪽은 완벽하게 미디엄 레어로 조리되어 육즙이 가득 차 있습니다. \n\n스테이크 위에 올려진 허브 버터는 신선한 맛과 크리미한 질감을 더해줍니다. 특히 파슬리와 같은 허브가 주는 상쾌함이 스테이크의 풍미를 한층 끌어올립니다. 부드럽고 살짝 짭짤한 맛이 조화를 이루어, 고기의 풍미가 강조됩니다.\n\n디스플레이된 아루굴라(루콜라)는 쌉쌀한 맛을 추가하여 전체적인 맛의 균형을 맞추며, 스테이크와의 조화를 잘 이룹니다.\n\n이 요리는 일반적으로 풀바디의 레드 와인과 잘 어울립니다. 특히 카베르네 소비뇽이나 말벡 같은 와인은 고기의 진한 맛과 버터의 크리미함을 보완하며, 잔잔한 탄닌이 부드러운 질감과 단맛을 더욱 강조해줍니다. \n\n이처럼 풍부하고 조화로운 맛의 조합은 식사 경험을 더욱 특별하게 만들어 줍니다.'

In [26]:
from langchain_core.runnables import RunnableLambda
runnable = RunnableLambda(describe_dish_flavor_chain)
response = runnable.invoke(query_2)
response

'이 요리는 부드러운 안심 스테이크로, 요리의 중심은 육즙이 풍부한 고기의 깊은 맛입니다. 스테이크의 겉은 잘 구워져 바삭한 크러스트가 형성되어 있으며, 내부는 연하고 분홍빛의 미디엄 레어로 조리되어 있습니다. \n\n### 맛 분석:\n- **Dominant Flavors**: 스테이크의 고소한 맛이 중심을 이루며, 고기의 자연적인 단맛도 느껴집니다. 마리네이드나 조리법에 따라 약간의 스파이스가 느껴질 수도 있습니다.\n- **Richness**: 고기는 풍부하고 기름진 질감이 배어 있어 한 입 먹을 때마다 입안에서 사르르 녹는 느낌을 줍니다.\n- **Herb Butter**: 버터 위에 올려진 허브는 신선함과 약간의 유산균적 산미를 더해주어 고기의 풍미와 잘 어울립니다. 허브의 향긋함이 스테이크의 본연의 맛을 강조해 줍니다.\n\n### 텍스처 및 아로마:\n- **Texture**: 스테이크의 부드러움은 씹었을 때의 쫄깃함과 함께 완벽한 조화를 이루며, 허브 버터의 크리미함이 곁들여져 전체적인 경험을 한층 증진시킵니다.\n- **Aroma**: 풍미가 가득한 고기는 구울 때 발생하는 탄내와 함께 상승하는 향긋한 허브의 향이 어우러져 미각을 자극합니다. \n\n### 음료 추천:\n이 요리와 함께 추천되는 와인은 풀바디의 레드 와인, 특히 까베르네 소비뇽이나 시라즈가 좋습니다. 이러한 와인은 스테이크의 풍부한 맛과 바디감에 잘 어울리며, 탄닌의 결합으로 각각의 맛이 더 풍부하게 느껴질 것입니다.'

In [18]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader("./winemag-data-130k-v2.csv")
docs = loader.load()

for i, doc in enumerate(docs[:3]):
    print(str(i), doc)

0 page_content=': 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia' metadata={'source': './winemag-data-130k-v2.csv', 'row': 0}
1 page_content=': 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Quinta dos Avidagos 2011 Avidagos Red (Douro)
varie

In [22]:
from langchain_pinecone import PineconeVectorStore
import os

vector_store = PineconeVectorStore.from_documents(
    docs, 
    embedding, 
    index_name='wine-reviews', 
    namespace='default'
)

In [24]:
results = vector_store.similarity_search(
    "이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다.", 
    k=5, 
    namespace='default'
)

results

[Document(id='85f135cb-d1bb-49dd-aafb-82bb5446f32a', metadata={'row': 55680.0, 'source': './winemag-data-130k-v2.csv'}, page_content=': 55680\ncountry: Italy\ndescription: Villa Novara presents easy aromas of fresh forest berry and almond paste. The wine is informal and sharp making it an ideal companion to pizza with mozzarella or pasta stuffed with ricotta cheese.\ndesignation: Villa Novare\npoints: 84\nprice: 16.0\nprovince: Veneto\nregion_1: Valpolicella Classico\nregion_2: \ntaster_name: \ntaster_twitter_handle: \ntitle: Bertani 2009 Villa Novare  (Valpolicella Classico)\nvariety: Corvina, Rondinella, Molinara\nwinery: Bertani'),
 Document(id='6cf56165-4693-4db1-bc0b-c18c8c191a30', metadata={'row': 66281.0, 'source': './winemag-data-130k-v2.csv'}, page_content=': 66281\ncountry: Italy\ndescription: A blend of 85% Sangiovese, 10% Alicante and 5% Ciliegiolo, this wine has earthy aromas that recall wet soil, leather and pressed blue flower petals. The juicy palate delivers fleshy bla

# search_wine

In [25]:
def search_wine(dish_flavor):
    results = vector_store.similarity_search(
        dish_flavor,
        k=5,
        namespace='default'
    )

    return {
        "dish_flavor": dish_flavor,
        "wine_reviews": "\n".join([doc.page_content for doc in results])
    }


In [27]:
taste_query = "이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다."
runnable = RunnableLambda(search_wine)
response = runnable.invoke(taste_query)
print(response['dish_flavor'])
print(response['wine_reviews'])

이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다.
: 55680
country: Italy
description: Villa Novara presents easy aromas of fresh forest berry and almond paste. The wine is informal and sharp making it an ideal companion to pizza with mozzarella or pasta stuffed with ricotta cheese.
designation: Villa Novare
points: 84
price: 16.0
province: Veneto
region_1: Valpolicella Classico
region_2: 
taster_name: 
taster_twitter_handle: 
title: Bertani 2009 Villa Novare  (Valpolicella Classico)
variety: Corvina, Rondinella, Molinara
winery: Bertani
: 66281
country: Italy
description: A blend of 85% Sangiovese, 10% Alicante and 5% Ciliegiolo, this wine has earthy aromas that recall wet soil, leather and pressed blue flower petals. The juicy palate delivers fleshy black cherry and raspberry. Notes of black pepper, espresso and Mediterranean herbs add interest.
designation: Ciabatta
points: 88
price: 25.0
province: Tuscany
region_1: Morellino di Scansano
region_2: 
taster_name: Kerin O

# recommand_wine

In [29]:
def recommand_wine(query):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
            Persona:

            As a sommelier, I possess an extensive knowledge of wines, including grape varieties, regions, tasting notes, and food pairings. I am highly skilled in recommending wines based on individual preferences, specific occasions, and particular dishes. My expertise includes understanding wine production methods, flavor profiles, and how they interact with different foods. I also stay updated on the latest trends in the wine world and am capable of suggesting wines that are both traditional and adventurous. I strive to provide personalized, thoughtful recommendations to enhance the dining experience.

            Role:

            1. Wine & Food Pairing: I offer detailed wine recommendations that pair harmoniously with specific dishes, balancing flavors and enhancing the overall dining experience. Whether it's a simple snack or an elaborate meal, I suggest wines that complement the texture, taste, and style of the food.
            2. Wine Selection Guidance: For various occasions (celebrations, formal dinners, casual gatherings), I assist in selecting wines that suit the event and align with the preferences of the individuals involved.
            3. Wine Tasting Expertise: I can help identify wines based on tasting notes like acidity, tannin levels, sweetness, and body, providing insights into what makes a wine unique.
            4. Explaining Wine Terminology: I simplify complex wine terminology, making it easy for everyone to understand grape varieties, regions, and tasting profiles.
            5. Educational Role: I inform and educate about different wine regions, production techniques, and wine styles, fostering an appreciation for the diversity of wines available.

            Examples:

            - Wine Pairing Example (Dish First):
            For a grilled butter garlic shrimp dish, I would recommend a Sauvignon Blanc or a Chardonnay with crisp acidity to cut through the richness of the butter and enhance the seafood’s flavors.

            - Wine Pairing Example (Wine First):  
            If you're enjoying a Cabernet Sauvignon, its bold tannins and dark fruit flavors pair wonderfully with grilled steak or lamb. The richness of the meat complements the intensity of the wine.

            - Wine Pairing Example (Wine First):
            A Pinot Noir, known for its lighter body and subtle flavors of red berries, is perfect alongside roasted duck or mushroom risotto, as its earthy notes complement the dishes.

            - Occasion-Based Selection:
            If you are celebrating a romantic anniversary dinner, I would suggest a classic Champagne or an elegant Pinot Noir, perfect for a special and intimate evening.

            - Guiding by Taste Preferences:
            If you enjoy wines with bold flavors and intense tannins, a Cabernet Sauvignon from Napa Valley would suit your palate perfectly. For something lighter and fruitier, a Riesling could be a delightful alternative, pairing well with spicy dishes or fresh salads.
        """),
        ("human", """
            와인 페이링 추천에 아래 요리/맛, 와인 리뷰만을 참고하여 한글로 답변해 주시기 바랍니다.

            요리/맛:
            {dish_flavor}

            와인 리뷰:
            {wine_reviews}
        """)
    ])

    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    
    return chain

# chain 연결

In [30]:
runnable_1 = RunnableLambda(describe_dish_flavor_chain)
runnable_2 = RunnableLambda(search_wine)
runnable_3 = RunnableLambda(recommand_wine)

chain = runnable_1 | runnable_2 | runnable_3

In [ ]:
# !pip install -qU grandalf

In [ ]:
# 음식이미지 링크 > 음식찾아서 음식맛 
# > 음식맛이 와인평에 있는 와인 검색  
# > 와인평 + 음식맛 
# > 소믈리에메시지
chain.get_graph().print_ascii()

+----------------------------------+ 
| describe_dish_flavor_chain_input | 
+----------------------------------+ 
                  *                  
                  *                  
                  *                  
           +------------+            
           | ChatOpenAI |            
           +------------+            
                  *                  
                  *                  
                  *                  
          +-------------+            
          | search_wine |            
          +-------------+            
                  *                  
                  *                  
                  *                  
           +------------+            
           | ChatOpenAI |            
           +------------+            
                  *                  
                  *                  
                  *                  
      +-----------------------+      
      | recommand_wine_output |      
      +-----

In [ ]:
response = chain.invoke(query_2)

response